In [6]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import scipy.io as io

%matplotlib inline

In [9]:
# algorithm to run collaborative representation-based classification on dataset

def collab_rep(X_train, X_test, Y_train, Y_test, lambda_weight=0):
    
    num_test_samples = len(Y_test)
    num_classifications = 15
    
    # least squares
    X = np.matrix(X_train)
    w = np.linalg.pinv(np.dot(X.T,X)) * (X.T) 
    
    predicted_label = np.zeros((num_test_samples,))
    
    # for each new vector
    for i in range(0,num_test_samples):
        test_ex = np.matrix(X_test[:,i]).T
        p = w * test_ex
        
        # compare the new vector to linear combinations of existing vectors by face
        dist = np.zeros((num_classifications,))
        for j in range(0,num_classifications):
            m = X[:,j*num_classifications:(j+1)*num_classifications]
            w_j = p[j*num_classifications:(j+1)*num_classifications]
            reconstructed = np.dot(m, w_j)
            dist[j] = np.linalg.norm(reconstructed - test_ex)
        
        # classify new vector according to minimum distance between the vector and reconstructed 
        predicted_label[i] = np.argmin(dist)
        
    # calculate the accuracy
    test_err = np.count_nonzero(predicted_label - Y_test)
    test_acc = 1-(test_err/100.0)
    
    return test_acc

In [12]:
# run the algorithm with k-fold cv 

# load YALE database from http://www.cad.zju.edu.cn/home/dengcai/Data/FaceData.html
# load data and transpose so that columns are faces
YALE = io.loadmat('Yale_64x64.mat')
fea = YALE['fea'].T
gnd = YALE['gnd'].T

# scale pixels to be [0,1]
maxValue = np.amax(fea)
fea = fea / maxValue

accuracy_list = []
for i in range(1,51,1):
    subset = io.loadmat('8Train/' + str(i) + '.mat') 
    trainIdx = subset['trainIdx'].flatten()
    testIdx = subset['testIdx'].flatten()
    
    # generate training and test data
    fea_Train = fea[:,trainIdx-1]; 
    fea_Test = fea[:,testIdx-1]; 

    gnd_Train = gnd[:,trainIdx-1]; 
    gnd_Test = gnd[:,testIdx-1]; 
    
    # get accuracy via collaborative representation
    accuracy = collab_rep(fea_Train, fea_Test, gnd_Train, gnd_Test, lambda_weight = 0)
    accuracy_list.append(accuracy)

# print(accuracy_list)
print("accuracy: ", np.mean(accuracy_list))

accuracy:  0.5512
